# Measure Distance with Moku

by Jason Ball, Liquid Instruments

jason@liquidinstruments.com

Created: 2024/12/20

Last edited: 2025/01/03

## Section 1: Import and Connect

In [3]:
### standard imports ###
### in this case we use tkinter for our updating GUI ### 
import tkinter as Tk
import numpy as np
import time

In [5]:
### import required instruments ###
### In this case, we are only using the Logic Analyzer in single-instrument mode ###
from moku.instruments import LogicAnalyzer

In [7]:
### Establish connection to the Moku ###
### This sets up the device as a Logic Analyzer / Pattern Generator in single-instrument mode. ###
### Input your device's IP address as a string. ###
### Setting force connect to be true will boot anyone using the device ### 
logan = LogicAnalyzer('[fe80::32e2:83ff:feb2:674c%11]', force_connect=True)

## Section 2: Power on range finder

In [10]:
### use the following command to enable the power supply, and set the voltage and current limits ###
### 150 mA is more than enough for our device ###   
logan.set_power_supply(1, enable=True, voltage=5, current=0.15) 

{'actual_current': 0.027484741210937524,
 'actual_voltage': 5.0625,
 'constant_current_mode': False,
 'constant_voltage_mode': True,
 'current_range': [0.0, 0.15],
 'enabled': True,
 'id': 1,
 'set_current': 0.15,
 'set_voltage': 5.0,
 'voltage_range': [-5.0, 5.0]}

## Section 3: Set up Logic Analyzer

In [13]:
### We need to set up the Logic Analyzer to read the serial data from Input 1. ###
logan.set_source('DigitalIO')

### The sequence is ~1 ms long, so we set up an appropriate time base ###
logan.set_timebase(-100e-6, 900e-6, roll_mode=False)

### Set up the protocol decoder on Ch1, bit 0. The format is UART, with a data format of 8 bits followed by one stop bits ###
### The Baud rate is 115200 ### 
logan.set_uart_decoder(channel=1, data_bit=0, data_width=8, uart_stop_width=1, uart_baud_rate=115200)

### The data is returned as a dictionary, containing raw and decoded data ###
### we look for the appropriate bit on pa1, which gives the distance ### 
logan.get_data(wait_reacquire=True, wait_complete=True)['pa1']

[{'data': 0, 'end': -9.480000000000011e-06, 'start': None, 'state': 'idle'},
 {'data': 89,
  'end': 7.317599999999999e-05,
  'start': -9.480000000000011e-06,
  'state': 'data'},
 {'data': 0,
  'end': 7.809599999999999e-05,
  'start': 7.317599999999999e-05,
  'state': 'idle'},
 {'data': 89,
  'end': 0.00015976800000000002,
  'start': 7.809599999999999e-05,
  'state': 'data'},
 {'data': 0,
  'end': 0.00016468800000000002,
  'start': 0.00015976800000000002,
  'state': 'idle'},
 {'data': 105,
  'end': 0.000247344,
  'start': 0.00016468800000000002,
  'state': 'data'},
 {'data': 0, 'end': 0.00025128, 'start': 0.000247344, 'state': 'idle'},
 {'data': 2, 'end': 0.000333936, 'start': 0.00025128, 'state': 'data'},
 {'data': 0, 'end': 0.000337872, 'start': 0.000333936, 'state': 'idle'},
 {'data': 26,
  'end': 0.00042052800000000004,
  'start': 0.000337872,
  'state': 'data'},
 {'data': 0,
  'end': 0.000424464,
  'start': 0.00042052800000000004,
  'state': 'idle'},
 {'data': 3, 'end': 0.00050712,

In [ ]:
### We set up a loop using tkinter to continuously update a display. ###
### The distance is acquired in every stage of the loop ### 
### Please see tkinter documentation for the exact nature of the commands ### 

def update_value(root, label):
    try:
        distance_in_cm = logan.get_data(wait_reacquire=True, wait_complete=True)['pa1'][5]['data']
    except:
        distance_in_cm = 1
    label1.config(text = f"Distance = {distance_in_cm} cm", font=('Arial', '144'),fg='white')
    root.after(20, update_value, root, label1)
    
root = Tk.Tk() 
label1 = Tk.Label(root, text = "Loading")
label1.pack()
root.after(1, update_value, root, label1)
root.mainloop()

2025-01-30 10:06:42.341 python[35130:651794] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-30 10:06:42.341 python[35130:651794] +[IMKInputSession subclass]: chose IMKInputSession_Modern
